## Estimate the activity of a simulated surface

#### Imports

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
sys.path.append('../scripts')
from functions import *

#### Simulate HEA surface

In [2]:
# Initialize a surface from a given stochiometry
dim_x, dim_y, split = 200, 200, [0.2 , 0.2 , 0.2 , 0.2 , 0.2 ]

HEA_surface = initialize_surface(dim_x, dim_y, metals, split)

# Predict energies on all sites for both adsorbates + mixed_sites
HEA_surface = precompute_binding_energies_SPEED(HEA_surface, dim_x, dim_y, models)

#### Estimate activity of each site

In [ ]:
# Make a function, that takes a surface in and returns it with a new
# matrix representing the activities on each site

# Decisions based on deltaG I guess
# Go through all indexes (x, y) and check for any negtive H neighb
# binding energies

In [13]:
def estimate_activity(surface, U):
    surface["COOH_G"] = surface["COOH_E"] + corrections["COOH"]
    surface["H_G"]    = surface["H_E"]    + corrections["H"]

    surface["Site_Activities"] = np.zeros([200, 200])

    for idx_x in range(dim_x):
        for idx_y in range(dim_y):
            # Check if any of the neighbouring hollow sites could house an H

            for x_diff, y_diff in [(0, 0), (-1, 0), (0, -1)]:
                neighbour_H_G = surface["H_G"][idx_x+x_diff, idx_y+y_diff]
                if neighbour_H_G < 0:
                    continue
            
            # If we get here it means no H + COOH disprop reac
            # Now use COOH_G to estimate activity

            surface["Site_Activities"][idx_x][idx_y] = per_site_current(surface["COOH_G"][idx_x][idx_y], U)

    return surface

def per_site_current(deltaG, U):
    deltaG_RLS = np.abs(deltaG+0.17)-0.17-U # This term will be 0 in the most optimal case
    kbT = 4.1164049935e-21
    j_k = 1* np.e**(-deltaG_RLS/kbT)
    return j_k

In [20]:
HEA_surface = estimate_activity(HEA_surface, U = -0.5)

In [22]:
HEA_surface["Site_Activities"]

0.0

In [19]:
HEA_surface["COOH_G"]

array([[1.7004739 , 1.2640857 , 1.6628963 , ..., 0.93822813, 0.39652085,
        1.3439863 ],
       [0.88499385, 1.3212154 , 0.85029376, ..., 0.8686634 , 1.6988672 ,
        1.2288053 ],
       [0.82666576, 1.5859829 , 1.2794973 , ..., 1.419919  , 0.91659236,
        0.9171337 ],
       ...,
       [0.87909913, 1.7064533 , 1.2674198 , ..., 1.319704  , 1.4416893 ,
        1.2700175 ],
       [1.6077666 , 1.3183583 , 1.6396664 , ..., 0.99981225, 1.365731  ,
        1.3966595 ],
       [1.2669193 , 0.5578434 , 1.3119998 , ..., 1.5039846 , 0.53246415,
        0.9345758 ]], dtype=float32)

In [10]:
kbT = 1.380649e-23 * 298.15
print(kbT)

4.1164049935e-21


In [7]:
np.zeros([200, 200])

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
HEA_surface.keys()

dict_keys(['atoms', 'stochiometry', 'ads_top', 'ads_hol', 'COOH_E', 'H_E', 'mixed_down', 'mixed_up_right', 'mixed_up_left', 'O_E', 'OH_E', 'H_V', 'O_V', 'COOH_V', 'OH_V', 'COOH_given_H_down', 'COOH_given_H_up_right', 'COOH_given_H_up_left', 'H_given_COOH_down', 'H_given_COOH_up_right', 'H_given_COOH_up_left'])

In [5]:
corrections["COOH"]

0.4950000000000001